# CAST
Collocated
Allong
Satellite
Track

### ToDo
- [ ] read model data into a xarray dataset
- [ ] read ADM data into a xarray dataset
- collocate model to ADM grid
  - [ ] lon/lat
  - [ ] time (not using right now)
  - [ ] altitude/elevation

# Conda environment at PPI/lustre

```bash
# load right (ana)conda module
module load aerocom/anaconda3-stable

# start notebook
jupyter lab --no-browser --ip=$HOSTNAME.met.no

```

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from glob import glob

for m in [np, pd, xr]:
    print(m.__name__, m.__version__)

numpy 1.15.4
pandas 0.23.4
xarray 0.10.9


# Read NetCDF files
list the emep forecast and adm orbit files for a given date

In [2]:
lustre = '/lustre/storeB/project/fou/kl/%s'
archive = dict(
    emepmodel = lustre%'emep/ModelRuns/ADM_Aeolus/EMEPmodel/CWF_12ST-%Y%m%d_hourInst.nc',
    topomodel = lustre%'emep/ModelRuns/ADM_Aeolus/EMEPmodel/MACC14_topo.nc',
    admaeolus = lustre%'admaeolus/data.rev.2A02/netcdf/2018-12/AE_OPER_ALD_U_N_2A_%Y%m%d*.DBL.nc',
)

date = pd.to_datetime('20181231')
ncfile = dict(
    emepmodel = date.strftime(archive['emepmodel']),
    topomodel = None, # waiting for topo file for MACC14 domain
    admaeolus = glob(date.strftime(archive['admaeolus'])),
)

for k, v in ncfile.items():
    if isinstance(v, str):
        print(k, v)
    elif isinstance(v, list):
        for f in v:
            print(k, f)


emepmodel /lustre/storeB/project/fou/kl/emep/ModelRuns/ADM_Aeolus/EMEPmodel/CWF_12ST-20181231_hourInst.nc
admaeolus /lustre/storeB/project/fou/kl/admaeolus/data.rev.2A02/netcdf/2018-12/AE_OPER_ALD_U_N_2A_20181231T053126033_005423993_002067_0001.DBL.nc
admaeolus /lustre/storeB/project/fou/kl/admaeolus/data.rev.2A02/netcdf/2018-12/AE_OPER_ALD_U_N_2A_20181231T001202031_005423993_002063_0001.DBL.nc
admaeolus /lustre/storeB/project/fou/kl/admaeolus/data.rev.2A02/netcdf/2018-12/AE_OPER_ALD_U_N_2A_20181231T130038029_005411999_002072_0001.DBL.nc
admaeolus /lustre/storeB/project/fou/kl/admaeolus/data.rev.2A02/netcdf/2018-12/AE_OPER_ALD_U_N_2A_20181231T195450027_004740011_002076_0001.DBL.nc
admaeolus /lustre/storeB/project/fou/kl/admaeolus/data.rev.2A02/netcdf/2018-12/AE_OPER_ALD_U_N_2A_20181231T225502032_005400005_002078_0001.DBL.nc
admaeolus /lustre/storeB/project/fou/kl/admaeolus/data.rev.2A02/netcdf/2018-12/AE_OPER_ALD_U_N_2A_20181231T113102030_005400006_002071_0001.DBL.nc
admaeolus /lustre/

## EMEP forecat
model `Z_MID` + `topography`

In [3]:
emep = xr.open_dataset(ncfile['emepmodel']).rename({"Z_MID":"alt"}).set_coords("alt")
if ncfile['topomodel']:
    topo = xr.open_dataset(ncfile['topomodel']).topography.isel(time=0)
    topo["lon"] = emep.lon
    topo["lat"] = emep.lat
    emep["alt"] += topo
emep

<xarray.Dataset>
Dimensions:         (ilev: 21, lat: 369, lev: 20, lon: 301, time: 25)
Coordinates:
  * lon             (lon) float64 -30.0 -29.75 -29.5 -29.25 ... 44.5 44.75 45.0
  * lat             (lat) float64 30.0 30.12 30.25 30.38 ... 75.75 75.88 76.0
  * lev             (lev) float64 0.1167 0.1528 0.1888 ... 0.9703 0.9838 0.9946
  * ilev            (ilev) float64 0.09869 0.1347 0.1708 ... 0.9784 0.9892 1.0
  * time            (time) datetime64[ns] 2018-12-31 ... 2019-01-01
    alt             (time, lev, lat, lon) float32 ...
Data variables:
    P0              float64 ...
    hyam            (lev) float64 ...
    hybm            (lev) float64 ...
    hyai            (ilev) float64 ...
    hybi            (ilev) float64 ...
    AOD_350nm       (time, lat, lon) float32 ...
    AOD_DUST_350nm  (time, lat, lon) float32 ...
    AOD_SS_350nm    (time, lat, lon) float32 ...
    AOD_SIA_350nm   (time, lat, lon) float32 ...
    AOD_500nm       (time, lat, lon) float32 ...
    AOD_550nm 

## ADM observations

In [4]:
adm = xr.open_dataset(ncfile['admaeolus'][0])
adm

<xarray.Dataset>
Dimensions:    (point: 10872)
Coordinates:
  * point      (point) int64 0 1 2 3 4 5 ... 10866 10867 10868 10869 10870 10871
Data variables:
    time       (point) datetime64[ns] ...
    ec355aer   (point) float64 ...
    bs355aer   (point) float64 ...
    latitude   (point) float64 ...
    longitude  (point) float64 ...
    altitude   (point) float64 ...
Attributes:
    product:              AE_OPER_ALD_U_N_2A_20181231T053126033_005423993_002...
    proc_stage:           N
    ref_doc:              AE-IF-DLR-L2A-004 03.05
    acquisition_station:  SVA_AS              
    proc_center:          APF   
    proc_time:            599556024.0
    software_ver:         ADM_L2aP/03.06
    baseline:             2A02                         
    sensing_start:        599549486.033888
    sensing_stop:         599554910.02745
    phase:                1
    cycle:                18
    rel_orbit:            56
    abs_orbit:            2067
    state_vector_time:    599549486.23

# CAST

### Discard ADM outside the forecast domain

In [5]:
domain = dict(
    time = emep.time[[0, -1]].values,
    latitude = emep.lat[[0, -1]].values,
    longitude = emep.lon[[0, -1]].values,
    altitude = [0.0, emep.alt.isel(lev=0).max().values],
)
domain

{'time': array(['2018-12-31T00:00:00.000000000', '2019-01-01T00:00:00.000000000'],
       dtype='datetime64[ns]'),
 'latitude': array([30., 76.]),
 'longitude': array([-30.,  45.]),
 'altitude': [0.0, array(15571.04394531)]}

In [6]:
in_range = lambda x, k: np.logical_and(x[k] >= domain[k][0], x[k] <= domain[k][1])
in_latlon = lambda x: np.logical_and(in_range(x, 'latitude'), in_range(x, 'longitude'))
in_domain = lambda x: np.logical_and(in_latlon(x), in_range(x, 'altitude'))
in_forecast=lambda x: np.logical_and(in_domain(x), in_range(x, 'time'))

mask = in_forecast(adm)
print('#points:   %6d'%adm.ec355aer.count())
print('#latlon:   %6d'%in_latlon(adm).sum())
print('#domain:   %6d'%in_domain(adm).sum())
print('#forecast: %6d'%mask.sum())

#points:     8137
#latlon:     1048
#domain:      930
#forecast:    930


### Filtered ADM

In [7]:
adm = adm.where(mask, drop=True)
adm.ec355aer

<xarray.DataArray 'ec355aer' (point: 930)>
array([0., 0., 0., ..., 0., 0., 0.])
Coordinates:
  * point    (point) int64 483 484 485 486 487 488 ... 1886 1887 1888 1889 1890
Attributes:
    long_name:      extinction @ 355nm
    standard_name:  volume_extinction_coefficient_in_air_due_to_ambient_aeros...
    units:          1/m

### Collocated forecast

In [8]:
collocate = lambda model, obs: model.load().sel(
    lon=obs.longitude,
    lat=obs.latitude,
#   alt=obs.altitude,
    time=obs.time,
    method='nearest'
)

In [9]:
%time ec355emep = collocate(emep.EXT_350nm, adm)

ec355emep

CPU times: user 9.59 s, sys: 584 ms, total: 10.2 s
Wall time: 12.3 s


<xarray.DataArray 'EXT_350nm' (point: 930, lev: 20)>
array([[0.000000e+00, 6.429294e-07, 7.472374e-07, ..., 1.275695e-05,
        1.205742e-05, 1.384416e-05],
       [0.000000e+00, 6.429294e-07, 7.472374e-07, ..., 1.275695e-05,
        1.205742e-05, 1.384416e-05],
       [0.000000e+00, 6.430570e-07, 7.474702e-07, ..., 1.284229e-05,
        1.224197e-05, 1.288030e-05],
       ...,
       [0.000000e+00, 1.082370e-06, 1.132606e-06, ..., 4.783900e-05,
        5.847721e-05, 6.758281e-05],
       [0.000000e+00, 1.082370e-06, 1.132606e-06, ..., 4.783900e-05,
        5.847721e-05, 6.758281e-05],
       [0.000000e+00, 1.082370e-06, 1.132606e-06, ..., 4.783900e-05,
        5.847721e-05, 6.758281e-05]], dtype=float32)
Coordinates:
    lon      (point) float64 24.5 24.5 24.25 24.25 24.25 ... 4.75 4.75 4.75 4.75
    lat      (point) float64 75.38 75.38 75.38 75.38 ... 30.25 30.25 30.25 30.25
  * lev      (lev) float64 0.1167 0.1528 0.1888 0.2271 ... 0.9703 0.9838 0.9946
    time     (point) datetim

In [10]:
find_nearest = lambda arr, v: (np.abs(arr - v)).argmin()
find_level = lambda p: find_nearest(ec355emep.alt.sel(point=p), adm.altitude.sel(point=p)).values

In [11]:
%time lev = xr.DataArray([find_level(p) for p in ec355emep.point], dims='point', coords={'point':ec355emep.point})

lev

CPU times: user 3.76 s, sys: 20 ms, total: 3.78 s
Wall time: 3.77 s


<xarray.DataArray (point: 930)>
array([ 0,  1,  2, ..., 13, 14, 15])
Coordinates:
  * point    (point) int64 483 484 485 486 487 488 ... 1886 1887 1888 1889 1890

In [12]:
%time ec355emep = ec355emep.isel(lev=lev)

ec355emep

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.01 ms


<xarray.DataArray 'EXT_350nm' (point: 930)>
array([0.000000e+00, 6.429294e-07, 7.474702e-07, ..., 8.169754e-05,
       8.227955e-05, 7.572624e-05], dtype=float32)
Coordinates:
    lon      (point) float64 24.5 24.5 24.25 24.25 24.25 ... 4.75 4.75 4.75 4.75
    lat      (point) float64 75.38 75.38 75.38 75.38 ... 30.25 30.25 30.25 30.25
    lev      (point) float64 0.1167 0.1528 0.1888 ... 0.8549 0.8932 0.9252
    time     (point) datetime64[ns] 2018-12-31T06:00:00 ... 2018-12-31T06:00:00
    alt      (point) float32 14266.854 12584.203 11244.549 ... 927.3764 638.9324
  * point    (point) int64 483 484 485 486 487 488 ... 1886 1887 1888 1889 1890
Attributes:
    long_name:           EXT_350nm
    units:               1/m
    class:               EXT:GROUP
    current_date_first:  [2018   12   31    0]
    numberofrecords:     25
    current_date_last:   [2019    1    1    0]